# Rolling the quantum dice multiple times

In the previous notebooks, we only rolled the dice _once_. Here, we will run the quantum program multiple times using the `trials` argument as to roll the dice multiple times!

Import the dependencies.

In [ ]:
%pylab inline
import pylab as pl
from functools import reduce

from pyquil.api import QPUConnection, QVMConnection, get_devices
from pyquil.quil import Program
from pyquil.gates import H, MEASURE

Create the QVM connection.

In [ ]:
qvm = QVMConnection()

## Roll an 8-sided quantum dice

To easily run this multiple times, let's create some functions: one to convert binary values to decimal values, and one to roll a dice _num_ times.

In [ ]:
def bin_to_dec(values: list):
    """Convert binary to decimal value"""
    return reduce(lambda x, y: 2*x + y, values, 0) + 1

def roll_d8(num: int = 10):
    """Roll an 8-sided dice <num> times"""
    dice = Program(H(0), H(1), H(2))
    roll_dice = dice.measure_all()
    result = qvm.run(roll_dice, trials=num)
    dice_values = [bin_to_dec(r) for r in result]

    return dice_values

Roll the dice once:

In [ ]:
roll_d8(1)

Example result: `[1]`

Roll the dice 10 times:

In [ ]:
roll_d8(10)

Example result: `[1, 5, 1, 3, 7, 8, 4, 4, 3, 3]`

Now, roll the dice 100 times and store the result in a variable.

In [ ]:
rolls = roll_d8(100)

We can then plot the result in a histogram to see the probability distribution. Try playing around with the number of rolls (N) and see how the probability distribution changes.

In [ ]:
def plot_hist(rolls, title="8-sided dice rolls", num_sides: int = 8):
    """Plot a histogram for 8-sided dice rolls"""
    pl.hist(rolls, bins=[i + .5 for i in range(num_sides+1)], rwidth=0.8)
    pl.ylabel("Number of rolls")
    pl.xlabel("Dice roll result")
    pl.title(title)

In [ ]:
plot_hist(rolls)

## Entangling two 8-sided quantum dice

Let's create two dice that are entangled! To do this, we first need to import the `CNOT` gate.

In [ ]:
from pyquil.gates import CNOT

To entangle two qubits, we can run the following program:

In [ ]:
entanglement = Program(H(0), CNOT(0, 1))
print(entanglement)

This program rotates one qubit into the superposition state, and then uses the _conditional not_ or `CNOT` gate to conditionally "flip" a second qubit, depending on the measurement outcome of the first. This means that if the first qubit returns |0>, the second qubit will stay in |0>, while if the first qubit returns |1>, the second qubit is flipped into |1>.

Let's run the program! You will see that because of quantum superposition, the two qubits will always give the same result.

In [ ]:
qvm.run(entanglement.measure_all(), trials=5)

Let's create a function that rolls two dice, and optionally entangles them.

In [ ]:
def roll_two_d8s(num: int = 10, entangled: bool = True):
    """Roll two entangled 8-sided dice <num> times"""
    # Create the first quantum dice state (qubits 0, 1 and 2).
    dice = Program(H(0), H(1), H(2))
    
    if entangled:
        # Entangle the dice to a second quantum dice (qubits 3, 4 and 5).
        two_dice = dice + Program(CNOT(0, 3), CNOT(1, 4), CNOT(2, 5))
    else:
        two_dice = dice + Program(H(3), H(4), H(5))

    # Roll the dice!
    roll_dice = two_dice.measure_all()
    result = qvm.run(roll_dice, trials=num)
    
    # Get the values for the dice by converting binary to decimal values.
    dice1_values, dice2_values = [], []
    for ( q0, q1, q2, q3, q4, q5 ) in result:
        # Dice 1 is encoded in qubits 0, 1 and 2:
        dice1_values += [bin_to_dec([q0, q1, q2])]

        # Dice 2 is encoded in qubits 3, 4 and 5:
        dice2_values += [bin_to_dec([q3, q4, q5])]
    
    # Return the result.
    return dice1_values, dice2_values

Plot the values to compare probability distributions, before and after entangling!

In [ ]:
rolls1, rolls2 = roll_two_d8s(100, entangled=False)
print("Number of times the two dice gave the same result:", sum(np.array(rolls1) == np.array(rolls2)))

pl.figure(figsize=(14, 4))
pl.subplot(121)
plot_hist(rolls1, "Dice A")
pl.subplot(122)
plot_hist(rolls2, "Dice B")

In [ ]:
rolls1, rolls2 = roll_two_d8s(100, entangled=True)
print("Number of times the two dice gave the same result:", sum(np.array(rolls1) == np.array(rolls2)))

pl.figure(figsize=(14, 4))
pl.subplot(121)
plot_hist(rolls1, "Dice A")
pl.subplot(122)
plot_hist(rolls2, "Dice B")